<span style="font-family:Lucida Bright;">

<hr style="border:2px solid black"> </hr>

<p style="margin-bottom:1cm"></p>

<center>
<font size="7"><b>Social Data Analysis and Visualization</b></font>
<p style="margin-bottom:1cm"></p>
<font size="6.8"><b>Final Project</b></font>   
<p style="margin-bottom:0.8cm"></p>
<font size="5.8"><b>Data Import and Cleaning</b></font>   
<p style="margin-bottom:0.8cm"></p>
<font size="3"><b>Wojciech Mazurkiewicz, DTU, 14 May 2021</b></font>
<br>
<font size="3"><b></b></font>

</center>

<p style="margin-bottom:0.7cm"></p>

<hr style="border:2px solid black"> </hr>

<hr style="border:2px solid black"> </hr>

# Initialization

## How to read this notebook

<span style="font-family:Arial;">

Please note that the pre-rendered outputs will first display properly when the notebook is __trusted__.

## Imports

In [49]:
%matplotlib inline

import bokeh.plotting as bplt
import calendar
import datetime
import folium
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import scipy.stats
import seaborn as sns
import urllib.request

from bokeh.io import output_file
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.models import Legend
from bokeh.models.ranges import FactorRange
from bokeh.models.sources import ColumnDataSource
from folium.map import FeatureGroup
from folium.plugins import HeatMap, HeatMapWithTime
from functools import reduce
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display
from IPython.display import Markdown
from IPython.display import YouTubeVideo
from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.image import NonUniformImage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from operator import itemgetter
from pathlib import Path
from scipy import stats
from sklearn.datasets import fetch_20newsgroups
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

##  Configuration

In [50]:
# Show bokeh figures in the notebook.
output_notebook()

# Below decide which output is shown below the cells.
InteractiveShell.ast_node_interactivity = "none"

# Define the format in which the numbers will be shown in
# the pandas dataframes.
pd.options.display.float_format = '{:,.2f}'.format

# Decide how to handle the "SettingWithCopyWarning" warning
pd.options.mode.chained_assignment = None  # default='warn'

# Set the maximum number of rows to show when displaying
# a Pandas dataframe.
pd.options.display.max_rows = 50

Loading BokehJS ...

## Paths

In [51]:
path_root = Path(
    r'C:\GDrive\DTU\Kurser\Social_Data_Analysis_and_Visualization_02806\final_project')
path_resources_root = path_root / 'resources'
path_data_root = path_resources_root / 'data'

path_data_citizenship_root = path_data_root / 'citizenship'
path_data_dwellings_root = path_data_root / 'dwellings'
path_data_geo_root = path_data_root / 'geo'
path_data_marital_status_root = path_data_root / 'marital_status'
path_data_without_district_info_root = path_data_root / 'without_district_information'

path_data_clean_root = path_data_root / 'clean'

## Function definitions

### General functions

In [52]:
# A function that will print a markdown text.
def printmd(string):
    display(Markdown(string))
    
def unique(list_):
    return list(set(list_))

### Load data

In [60]:
# A function that returns a dict of object attributes.
def get_obj_attributes(obj):
    return {attribute_name: getattr(obj, attribute_name)
            for attribute_name in dir(obj)
            if (not attribute_name.startswith('__')
                and not callable(getattr(obj, attribute_name)))}


class CphData:

    def __init__(self):
        # Country of origin (no distric info)
        df_country_of_origin = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_country_of_origin_without_district.pkl')
        df_country_of_origin.name = 'Country of origin'
        self.country_of_origin = df_country_of_origin

        # Citizenship
        df_citizenship = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_citizenship.pkl')
        df_citizenship.name = 'Citizenship'
        self.citizenship = df_citizenship

        # Marital statusf
        df_marital_status = pd.read_pickle(
            path_data_clean_root / 'cph_population_by_marital_status.pkl')
        df_marital_status.name = 'Marital status'
        self.marital_status = df_marital_status

        # Family type and children
        df_family_type_and_children = pd.read_pickle(path_data_clean_root /
                                                     'cph_population_by_family_type_and_number_of_chidren.pkl')
        df_family_type_and_children.name = 'Family type and children'
        self.family_type_and_children = df_family_type_and_children

        # Income
        df_income = pd.read_pickle(path_data_clean_root / 'cph_income.pkl')
        df_income.name = 'Income'
        self.income = df_income

        # Life span
        df_life_span = pd.read_pickle(
            path_data_clean_root / 'cph_life_span.pkl')
        df_life_span.name = 'Life span'
        self.life_span = df_life_span

        # Population movement
        df_population_movement = pd.read_pickle(
            path_data_clean_root / 'cph_population_movement.pkl')
        df_population_movement.name = 'Population movement'
        self.population_movement = df_population_movement

        # Dwellings
        df_dwellings = pd.read_pickle(
            path_data_clean_root / 'cph_dwellings.pkl')
        df_dwellings.name = 'Dwellings'
        self.dwellings = df_dwellings

    # Gets dataframes with district information:
    def get_dataframes_with_district(self):
        return [df
                for df in self.get_all_dataframes()
                if 'District' in df.columns]

    # Gets all the dataframes.
    def get_all_dataframes(self):
        return get_obj_attributes(self).values()

    # Gets the names of all columns in all datasets.
    def get_all_column_names(self):
        # Initialize the list of all columns from all the dataframes:
        all_columns = list()

        # Get all columns.
        for df in self.get_all_dataframes():
            all_columns += df.columns.to_list()

        return list(sorted(set(all_columns)))

    # Displays the dataframes with specified names.
    def display_dataframes(self, names=None):
        for attribute_name, df in get_obj_attributes(self).items():
            if (names is None) or (df.name in names):
                display(df)

    # Displays the names of the dataframes in the object.
    def display_dataframe_names(self):
        # Display all the dataframes.
        df_names = [df.name for df in self.get_dataframes()]
        printmd(f'The dataframes in the object are:')
        printmd(f'***{"***, ***".join(df_names)}***')

### Plotting functions

In [54]:
# A function that applies default formatting to an axes.
def format_axes(axes: plt.Axes,
                keep_box=False):
    if not keep_box:
        axes.spines['top'].set_color('white')
        axes.spines['right'].set_color('white')

    axes.set_facecolor("white")


# A function that applies default formatting to annotation
# of an axes.
def format_axes_annotation(axes: plt.Axes):
    axes.xaxis.label.set_fontsize(14)
    axes.yaxis.label.set_fontsize(14)
    axes.title.set_fontsize(16)


# A function for creating common x-label for the figure.
def figure_x_label(figure: plt.Figure,
                   label: str,
                   y_position=0.04,
                   font_size=16):
    figure.text(0.5, y_position, label,
                ha='center',
                fontdict={'size': font_size})


# A function for creating common y-label for the figure.
def figure_y_label(figure: plt.Figure,
                   label: str,
                   x_position=0.04,
                   font_size=16):
    figure.text(x_position, 0.5, label,
                va='center',
                rotation='vertical',
                fontdict={'size': font_size})


# A function that draws a horizontal line across the entire axes.
def draw_threshold(value: float,
                   axes: plt.Axes,
                   linewidth=1,
                   linestyle='-',
                   color=None,
                   title=None):
    
    # Get axes limits and ranges.
    x_min, x_max = axes.get_xlim()
    x_range = x_max - x_min
    y_min, y_max = axes.get_ylim()
    y_range = y_max - y_min
    
    # Plot the threshold line.
    axes.plot([x_min, x_max], [value, value],
              linewidth=1,
              linestyle='-',
              color=color)
    
    # Write a title above the threshold line
    if title is not None:
        axes.text(x_min + 0.01 * x_range,
                  value + 0.02 * y_range,
                  title)

###  Dataframe functions

In [55]:
# A function for balancing a dataframe so that the number of rows
# containing each value present in the designated column will be the same.
def balance_dataframe(df: pd.DataFrame, column_name):
    # Get the number of crimes for the least frequent crime.
    lowest_frequency = df['Category'].value_counts().min()

    # Create an empty dataframe for storing the balanced data
    df_balanced = pd.DataFrame()

    # For each value in column, randomly choose the number of samples
    # that corresponds to the least frequent value in the column.
    for value in df[column_name].unique():
        df_balanced = df_balanced.append(
            df
            .loc[df[column_name] == value]
            .sample(lowest_frequency)
        )

    return df_balanced


# A function that evaluates a dictionary of models on data from
# a pandas dataframe.
def evaluate_models(models: dict,
                    df: pd.DataFrame,
                    predictor_labels: list,
                    target_label: str,
                    test_size=0.33):

    # Get the dataset.
    X = df.loc[:, predictor_labels].values
    y = df.loc[:, target_label].values

    # Split the dataset into a test and training set.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=32)

    # Fit the models to the data.
    for model_name, model in models.items():

        # Print the name of the model.
        printmd(f'*__{model_name}:__*')

        # Train the model on the training set.
        model.fit(X_train, y_train)

        # Get the predictions on the test set.
        predictions = model.predict(X_test)

        # Print the classification report.
        print(classification_report(y_test, predictions,
                                    zero_division=0,
                                    digits=4))

        
# A function that sorts the columns in alphabethical order
# and puts the user-chosen columns first
def df_sort_columns(df: pd.DataFrame,
                    first_columns=['Year',
                                   'Quarter',
                                   'District',
                                   'Sex',
                                   'Age']):
    
    # Define a function that moves the chosen element to the
    # front of the list.
    def move_to_front(element, list_):
        if element in list_:
            list_.insert(0, list_.pop(list_.index(element)))
    
    # Make sure that the columns that are to be put in front
    # are represented as a list.
    if first_columns is None:
        first_columns = list()
    
    # Sort the columns in alphabetical order.
    sorted_columns = list(df.columns)
    sorted_columns.sort()
    
    # Move the user-chosen columns to the front.
    for column in first_columns[::-1]:
        move_to_front(column, sorted_columns)
    
    # Assign the ordered columns to the dataframe.
    df = df[sorted_columns]

    return df


###  Data cleaning functions

In [56]:
# Define a function that loads a dataframe from
# kk.statistikbank.dk
def load_cph_df(path_csv):
    return pd.read_csv(path_csv,
                       sep='\t',
                       skiprows=0,
                       encoding='windows-1252')


# A function that loads a dataframe from multiple files.
def load_split_dataframe(paths_csv):

    # Load the first two dataframes.
    df_1 = load_cph_df(paths_csv[0])
    df_2 = load_cph_df(paths_csv[1])

    # Extract common columns.
    common_columns = list(set(df_1.columns)
                          & set(df_2.columns))

    # Load the whole dataframe.
    for idx, path_csv in enumerate(paths_csv):
        # Merge the dataframes.
        if idx == 0:
            df = load_cph_df(path_csv)
        else:
            df = df.merge(load_cph_df(path_csv),
                          left_on=common_columns,
                          right_on=common_columns)

    # Return the merged dataframe.
    return df


# A function that cleans the data about copenhagen district
def clean_districts(df: pd.DataFrame):

    # Continue only if the dataframe contains the column: "district"
    if 'district' not in df.columns:
        return df

    df = df.copy()

    # Define the dame of the column containing info about districts.
    district_column_name = 'district'

    # Extract the district names.
    districts = df[district_column_name]

    # Initialize the list for storing district types.
    district_types = list()

    # Define valid districts.
    valid_districts = ['Indre By', 'Østerbro', 'Nørrebro', 'Vesterbro/Kongens Enghave', 'Valby',
                       'Vanløse', 'Brønshøj-Husum', 'Bispebjerg', 'Amager Øst', 'Amager Vest']

    # Define valid districts.
    valid_polling_areas = ['1. Østerbro', '1. Øst', '1. Nord', '1. Syd', '1. Vest', '1. Nordvest',
                           '2. Sundbyvester', '2. Nord', '2. Syd', '2. Øst', '2. Vest', '3. Indre By',
                           '3. Nord', '3. Syd', '3. Øst',
                           '4. Sundbyøster', '4. Nord', '4. Syd', '4. Øst',
                           '5. Nørrebro', '5. Nord', '5. Nørrebrohallen',
                           '5. Syd', '5. Øst', '5. Vest', '5. Nordvest',
                           '6. Bispebjerg', '6. Vest', '6. Nord', '6. Øst', '6. Syd',
                           '7. Brønshøj', '7. Nord', '7. Syd', '7. Øst', '7. Vest',
                           '7. Nordvest', '7. Katrinedal', '7. Kirkebjerg', '7. Vanløse',
                           '8. Valby', '8. Nord', '8. Syd', '8. Vest', '8. Sydøst', '8. Midt',
                           '9. Vesterbro', '9. Nord', '9. Syd',
                           '9. Øst', '9. Vest', '9. Midt', '9. Sydhavn']

    # Assign type to each district.
    for district in districts:
        if (('District' in district) | (district in valid_districts)):
            district_types.append('District')

        elif (('Polling area' in district) | (district in valid_polling_areas)):
            district_types.append('Polling area')

        elif 'Copenhagen total' in district:
            district_types.append('Entire Copenhagen')

        else:
            district_types.append('Unknown')

    # Insert the column "district type" next to the district column.
    idx_district_column = df.columns.to_list().index(district_column_name)
    df.insert(idx_district_column + 1, 'district type', district_types)

    # Clean the names in the district - column.
    for token in ['District -', 'Polling area -']:
        df[district_column_name] = (
            df[district_column_name]
            .str.replace(token, '')
            .str.strip()
        )

    return df


# A function that cleans all the information about the time in the
# dataframes for Copenhagen.
def clean_years(df: pd.DataFrame,
                non_year_columns=None,
                value_name='Value'):

    if non_year_columns is not None:
        # If "non_year_columns" is an integer, regard the first n
        # columns represented by the integer as non-year columns
        if isinstance(non_year_columns, int):
            non_year_columns = df.columns[:non_year_columns]
    else:
        non_year_columns = [column for column in df.columns.to_list()
                            if column[0] not in ['1', '2']]

    # Create a row for each Year and Quarter.
    df = df.melt(
        id_vars=non_year_columns,
        var_name="Time",
        value_name=value_name
    )

    # If the time is represented by a year and a quarter,
    # create columns "Year" and "Quarter" from the column "Time".
    if 'Q' in df.loc[0, 'Time']:
        df[['Year', 'Quarter']] = (
            df
            .pop('Time').str.split('Q', 2, expand=True)
            .astype(int)
        )

    # If year is presented as "XXXX:YYYY", take the second value.
    elif ':' in df.loc[0, 'Time']:
        df['Year'] = (
            df
            .pop('Time')
            .str.split(':').str[-1]
            .astype(int)
        )

    # Otherwise, transform the year into an integer
    else:
        df['Year'] = df.pop('Time').astype(int)

    return df


# A fuction that removes all other quarters of the year than last.
def choose_latest_quarter(df: pd.DataFrame, value_name):
    # Run only if the dataframe has a column "Age"

    if 'Quarter' not in df.columns:
        return df

    return (
        df
        .sort_values(by=['Year', 'Quarter'])
        .groupby([column for column in df.columns
                  if column not in ['Quarter', value_name]],
                 as_index=False)
        .last()
    )


# A function that perfoms standard cleaning of a dataframe
# from kk.statistikbank.dk
def clean_cph_dataframe(df, value_name='Value', df_name=''):
    # Clean district information.
    df = clean_districts(df)

    # Clean time information.
    df = clean_years(df, value_name=value_name)

    # Capitalize column names.
    df.columns = [column.capitalize() for column in df.columns]

    # Standardize age intervals
    df_standardize_age(df, value_name)

    # Choose only the last available quarter of the year.
    df = choose_latest_quarter(df, value_name)  
    
    # Order the columns.
    df = df_sort_columns(df)
    
    # Name the dataframe.
    df.name = df_name

    return df


# A function that displays all basic stats about the dataframe.
def show_stats(df):
    # Show data types.
    display(df.dtypes.to_frame('Data types'))

    # Show missing values.
    display(
        df.isna()
        .sum()
        .to_frame('Number of missing values')
    )

    # Describe values in columns.
    display(df.describe(include='all'))


# A function that unifies age intervals in the databases from
# from Københavns Kommune:
def df_standardize_age(df: pd.DataFrame, sum_column):

    # Run only if the dataframe has a column "Age"
    if 'Age' not in df.columns:
        return df

    # Use 10-year intervals for age: Create the mapping.
    mapping_5_to_10_years = dict()
    for interval_min in range(0, 90, 10):
        map_from_low = f'{interval_min}-{interval_min + 4} years'
        map_from_high = f'{interval_min + 5}-{interval_min + 9} years'
        map_to = f'{interval_min}-{interval_min + 9}'

        mapping_5_to_10_years[map_from_low] = map_to
        mapping_5_to_10_years[map_from_high] = map_to

    # Define the mapping for all the possible versions of the 90+ age.
    mapping_5_to_10_years['90-99 + years'] = '90+'
    mapping_5_to_10_years['95-99 years'] = '90+'
    mapping_5_to_10_years['90-94 years'] = '90+'
    mapping_5_to_10_years['100 years and over'] = '90+'
    mapping_5_to_10_years['95+years'] = '90+'

    # Apply the mapping to the column "Age".
    df['Age'] = df['Age'].map(mapping_5_to_10_years)

    # Sum the values in the new bins. First, get the names
    # of all the columns that are NOT the column which will summed up.
    non_sum_columns = [column for column in df.columns
                       if column != sum_column]

    # Sum the values together in the age bins.
    df[sum_column] = (
        df
        .groupby(non_sum_columns)
        .transform('sum')
    )

    # Drop duplicate rows
    df.drop_duplicates(ignore_index=True, inplace=True)

    return df

<hr style="border:2px solid black"> </hr>

# Load data

In [62]:
# Load the clean data.
cph_data = CphData()   

# Display all dataframes:
cph_data.display_dataframes()


,Year,Quarter,District,Sex,Age,Citizenship,District type,Number of people
0,1980,1,1. Nord,Men,0-9,Denmark,Polling area,265
1,1981,1,1. Nord,Men,0-9,Denmark,Polling area,278
2,1982,1,1. Nord,Men,0-9,Denmark,Polling area,265
3,1983,1,1. Nord,Men,0-9,Denmark,Polling area,249
4,1984,1,1. Nord,Men,0-9,Denmark,Polling area,249
...,...,...,...,...,...,...,...,...
157435,2016,4,Østerbro,Women,90+,Western countries,District,3
157436,2017,4,Østerbro,Women,90+,Western countries,District,4
157437,2018,4,Østerbro,Women,90+,Western countries,District,5
157438,2019,4,Østerbro,Women,90+,Western countries,District,6


,Year,Quarter,Sex,Age,Country of origin,Number of people
0,2008,4,Men,0-9,Abu Dhabi,0
1,2009,4,Men,0-9,Abu Dhabi,0
2,2010,4,Men,0-9,Abu Dhabi,0
3,2011,4,Men,0-9,Abu Dhabi,0
4,2012,4,Men,0-9,Abu Dhabi,0
...,...,...,...,...,...,...
61875,2016,4,Women,90+,Zimbabwe,0
61876,2017,4,Women,90+,Zimbabwe,0
61877,2018,4,Women,90+,Zimbabwe,0
61878,2019,4,Women,90+,Zimbabwe,0


,Year,District,District type,Dwelling ownership,Total square meters occupied dwellings
0,1991,Copenhagen total,Entire Copenhagen,Owner-occupied,"4,667,001.00"
1,1991,Indre By,District,Owner-occupied,"566,699.00"
2,1991,Østerbro,District,Owner-occupied,"635,362.00"
3,1991,Nørrebro,District,Owner-occupied,"298,278.00"
4,1991,Vesterbro/Kongens Enghave,District,Owner-occupied,"273,584.00"
...,...,...,...,...,...
11899,2021,9. Syd,Polling area,Unknown,0.00
11900,2021,9. Øst,Polling area,Unknown,0.00
11901,2021,9. Vest,Polling area,Unknown,0.00
11902,2021,9. Midt,Polling area,Unknown,0.00


,Year,Quarter,District,District type,Family type,Number of children,Number of people
0,1998,1,1. Nord,Polling area,Children below 18 years not living with parents,0 children,5
1,1999,1,1. Nord,Polling area,Children below 18 years not living with parents,0 children,10
2,2000,1,1. Nord,Polling area,Children below 18 years not living with parents,0 children,17
3,2001,1,1. Nord,Polling area,Children below 18 years not living with parents,0 children,19
4,2002,4,1. Nord,Polling area,Children below 18 years not living with parents,0 children,26
...,...,...,...,...,...,...,...
36795,2016,4,Østerbro,District,Single women,More than 3 children,21
36796,2017,4,Østerbro,District,Single women,More than 3 children,20
36797,2018,4,Østerbro,District,Single women,More than 3 children,21
36798,2019,4,Østerbro,District,Single women,More than 3 children,19


Unit,Year,District,Sex,Amount of income (kr.),Average income for people with the type of income (kr.),District type,People with the type of income (number)
0,1987,1. Nord,Men,480298,190217,Polling area,2525
1,1987,1. Nord,Women,457876,123617,Polling area,3704
2,1987,1. Nordvest,Men,637196,145246,Polling area,4387
3,1987,1. Nordvest,Women,608784,108402,Polling area,5616
4,1987,1. Syd,Men,844061,152330,Polling area,5541
...,...,...,...,...,...,...,...
4285,2019,Vanløse,Women,5566365,319410,District,17427
4286,2019,Vesterbro/Kongens Enghave,Men,11307215,381678,District,29625
4287,2019,Vesterbro/Kongens Enghave,Women,9399658,319325,District,29436
4288,2019,Østerbro,Men,13865000,448024,District,30947


,Year,District,Average life span,District type
0,2009,Indre By,79.40,District
1,2009,Østerbro,78.40,District
2,2009,Nørrebro,72.30,District
3,2009,Vesterbro/Kongens Enghave,73.50,District
4,2009,Valby,75.90,District
...,...,...,...,...
127,2020,Brønshøj-Husum,78.50,District
128,2020,Bispebjerg,77.80,District
129,2020,Amager Øst,81.00,District
130,2020,Amager Vest,78.70,District


,Year,Quarter,District,Sex,Age,District type,Marital status,Number of people
0,1974,1,1. Nord,Men,90+,Polling area,Divorced,0.00
1,1975,1,1. Nord,Men,90+,Polling area,Divorced,1.00
2,1976,1,1. Nord,Men,90+,Polling area,Divorced,1.00
3,1977,1,1. Nord,Men,90+,Polling area,Divorced,0.00
4,1978,1,1. Nord,Men,90+,Polling area,Divorced,1.00
...,...,...,...,...,...,...,...,...
24059,2016,4,Østerbro,Women,90+,District,Widowed,235.00
24060,2017,4,Østerbro,Women,90+,District,Widowed,220.00
24061,2018,4,Østerbro,Women,90+,District,Widowed,211.00
24062,2019,4,Østerbro,Women,90+,District,Widowed,208.00


,Year,District,Deaths,District type,Emigrated,Immigrated,Internal migration,Internal migration from other districts,Internal migration to to other districts,Internal migratrion from danish municipalities,Internal migratrion to danish municipalities,Live births,Natural increase,Netmigration,Netmigration Copenhagen,Netmigration Denmark
0,1975,1. Nord,134,Polling area,122,84,158,663,744,562,668,93,-41,-38,-81,-106
1,1976,1. Nord,129,Polling area,98,88,110,622,681,496,637,73,-56,-10,-59,-141
2,1977,1. Nord,111,Polling area,90,92,127,637,633,446,564,72,-39,2,4,-118
3,1978,1. Nord,127,Polling area,72,77,111,652,638,465,499,73,-54,5,14,-34
4,1980,1. Nord,128,Polling area,115,107,99,686,558,587,501,75,-53,-8,128,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,2016,Østerbro,461,District,1877,2905,3788,5533,5835,5576,5467,1267,806,1028,-302,109
2579,2017,Østerbro,475,District,2006,3000,3855,5642,5918,5818,5904,1244,769,994,-276,-86
2580,2018,Østerbro,463,District,2235,3017,3709,5310,5963,4930,5726,1251,788,782,-653,-796
2581,2019,Østerbro,443,District,2277,3018,3813,5493,6120,5255,5699,1303,860,741,-627,-444


<hr style="border:2px solid black"> </hr>

# Data columns available the datasets

We know that we have 1 dataset without the district information: the one that describes the total population of Copenhagen by the country of origin. We will treat this dataset separately.

Now, let's see how much information the dataset containing district information have in common:

## Common columns

In [64]:
# Get the names of all columns in all tables.
all_columns = cph_data.get_all_column_names()

# Get all dataframes containing district information.
all_dataframes_with_district = cph_data.get_dataframes_with_district()

# Get a dict containing the names of dataframes that
# contain a given column.
datasets_with_column = {column: [df.name for df in all_dataframes_with_district
                                 if column in df.columns]
                        for column in all_columns}

# Get the column names sorted by the number of datasets in which they appear.
columns_by_popularity = sorted(datasets_with_column,
                               key=lambda column: len(
                                   datasets_with_column[column]),
                               reverse=True)

# Create dictionary sorted by the the popularity of the column
# (from most to least popular).
datasets_with_column_sorted = {column: datasets_with_column[column]
                               for column in columns_by_popularity}

# Print out the most popular columns.
for column_name, df_names in datasets_with_column_sorted.items():
    print()
    printmd(f'Column **_{column_name}_** occurs in {len(df_names)} '
            f'out of {len(all_dataframes_with_district)} dataframes:')
    print(df_names)

Column **_District_** occurs in 7 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Life span', 'Marital status', 'Population movement']



Column **_District type_** occurs in 7 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Life span', 'Marital status', 'Population movement']



Column **_Year_** occurs in 7 out of 7 dataframes:

...

['Income']



Column **_Total square meters occupied dwellings_** occurs in 1 out of 7 dataframes:

['Dwellings']



Column **_Country of origin_** occurs in 0 out of 7 dataframes:

[]


We can use **Year**, **District**, and **District Type** as basis of a superdataset, where we cross informations from various datasets.

## Steamline districts

For simplicity, it might be beneficial to use only one district type. Let's see which ones we have available:

In [11]:
# Get the names of all columns in all tables.
all_columns = cph_data.get_all_column_names()

# Get all dataframes containing district information.
all_dataframes_with_district = cph_data.get_dataframes_with_district()

# Get all district types.
all_district_types = list()
for df in cph_data.get_dataframes_with_district():
    all_district_types += list(df['District type'].unique())

all_district_types = unique(all_district_types)

# Display the district types.
printmd('All district types:')
for district in all_district_types:
    district_string = '***, ***'.join(all_district_types)

printmd('***' + '***, ***'.join(all_district_types) +  '***')

All district types:

***District***, ***Polling area***, ***Entire Copenhagen***

***Entire Copenhagen*** is not a district, but a summation of all districts in Copenhagen. ***District*** and ***Polling area*** are real districts - ***District*** is of lower resolution and ***Polling area*** is of higher resolution.

In [12]:
# Get all dataframes containing district information.
all_dataframes_with_district = cph_data.get_dataframes_with_district()

# Get all Polling Areas.
all_polling_areas = list()
for df in cph_data.get_dataframes_with_district():
    all_polling_areas += list(df.loc[df['District type'] == 'Polling area', 'District'].unique())
all_polling_areas = unique(all_polling_areas)

# Get all Districts.
all_districts = list()
for df in cph_data.get_dataframes_with_district():
    all_districts += list(df.loc[df['District type'] == 'District', 'District'].unique())
all_districts = unique(all_districts)

# Show the Polling Areas and Districts:
printmd(f'There are **{len(all_polling_areas)}** Polling Areas in Copenhagen:')
printmd(f'*{", ".join(sorted(all_polling_areas))}*')
printmd(f'There are **{len(all_districts)}** Districts in Copenhagen:')
printmd(f'*{", ".join(sorted(all_districts))}*')

There are **53** Polling Areas in Copenhagen:

*1. Nord, 1. Nordvest, 1. Syd, 1. Vest, 1. Øst, 1. Østerbro, 2. Nord, 2. Sundbyvester, 2. Syd, 2. Vest, 2. Øst, 3. Indre By, 3. Nord, 3. Syd, 3. Øst, 4. Nord, 4. Sundbyøster, 4. Syd, 4. Øst, 5. Nord, 5. Nordvest, 5. Nørrebro, 5. Nørrebrohallen, 5. Syd, 5. Vest, 5. Øst, 6. Bispebjerg, 6. Nord, 6. Syd, 6. Vest, 6. Øst, 7. Brønshøj, 7. Katrinedal, 7. Kirkebjerg, 7. Nord, 7. Nordvest, 7. Syd, 7. Vanløse, 7. Vest, 7. Øst, 8. Midt, 8. Nord, 8. Syd, 8. Sydøst, 8. Valby, 8. Vest, 9. Midt, 9. Nord, 9. Syd, 9. Sydhavn, 9. Vest, 9. Vesterbro, 9. Øst*

There are **11** Districts in Copenhagen:

*Amager Vest, Amager Øst, Bispebjerg, Brønshøj-Husum, Indre By, Nørrebro, Unlocated, Valby, Vanløse, Vesterbro/Kongens Enghave, Østerbro*

Note that one of the districts is ***Unlocated***.

Let' now take a look at which datasets have which dataset type:

In [13]:
datasets_with_district_type = {district_type: [df.name
                                               for df in all_dataframes_with_district
                                               if district_type in df['District type'].values]
                               for district_type in all_district_types}

# Get the district names sorted by the number of datasets in which they appear.
district_types_by_popularity = sorted(datasets_with_district_type,
                                      key=lambda column: len(
                                          datasets_with_district_type[column]),
                                      reverse=True)

# Create dictionary sorted by the the popularity of the district type.
# (from most to least popular).
datasets_with_district_type_sorted = {column: datasets_with_district_type[column]
                                      for column in district_types_by_popularity}

# Print out the most popular columns.
for column_name, df_names in datasets_with_district_type_sorted.items():
    print()
    printmd(f'Dataset type **_{column_name}_** occurs in {len(df_names)} '
            f'out of {len(all_dataframes_with_district)} dataframes:')
    print(df_names)

Dataset type **_District_** occurs in 7 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Life span', 'Marital status', 'Population movement']



Dataset type **_Polling area_** occurs in 6 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Marital status', 'Population movement']



Dataset type **_Entire Copenhagen_** occurs in 5 out of 7 dataframes:

['Citizenship', 'Dwellings', 'Family type and children', 'Income', 'Marital status']


So, if we use ***District*** as an area, we will be able to divide Copenhagen in 10 districts and use all of our datasets containing district informattion.

If we use ***Polling area***, we will be able to divide Copenhagen in 52 districts and use all of our datasets except the one containing information about *Life span*.

It is a bit of a bummer, because I would like to use the high resolution, but I am also very interested in taking a look on how life span correlates with all the other factors. Therefore, at least as a starting point, I will use ***District***.

Let's create an object for our superset, use only districts of type: ***District*** and drop the column: ***District type***, which becomes superficial.

In [65]:
# Initialize the dataset that will contain the cross of all the
# dataset available in this project.
cph_data_superset = CphData()

for attribute_name, df in get_obj_attributes(cph_data_superset).items():
    # Delete all the datasets from the object that do not contain
    # the district data
    if 'District' not in df.columns:
        delattr(cph_data_superset, attribute_name)
    
    # Limit the data to District Type: District and
    # remove the column: District type
    else:
        setattr(
            cph_data_superset,
            attribute_name,
            (df
             .loc[df['District type'] == 'District']
             .drop(['District type'], axis=1))
        )

# Display all dataframes:
display(cph_data_superset.i)

Unit,Year,District,Sex,Amount of income (kr.),Average income for people with the type of income (kr.),People with the type of income (number)
106,1987,Amager Vest,Men,2368858,138659,17084
107,1987,Amager Vest,Women,2056151,103898,19790
108,1987,Amager Øst,Men,2614378,140520,18605
109,1987,Amager Øst,Women,2191694,103348,21207
110,1987,Bispebjerg,Men,2127197,128679,16531
...,...,...,...,...,...,...
4285,2019,Vanløse,Women,5566365,319410,17427
4286,2019,Vesterbro/Kongens Enghave,Men,11307215,381678,29625
4287,2019,Vesterbro/Kongens Enghave,Women,9399658,319325,29436
4288,2019,Østerbro,Men,13865000,448024,30947


## Remove quarter

In [70]:
for attribute_name, df in get_obj_attributes(cph_data_superset).items():
    # Delete all the datasets from the object that do not contain
    # the district data.
    if 'Quarter' in df.columns:
        df.drop(['Quarter'], axis=1, inplace=True)
    
# Display all dataframes:
display(cph_data_superset.family_type_and_children)

,Year,District,Family type,Number of children,Number of people
1219,1998,Amager Vest,Children below 18 years not living with parents,0 children,82
1220,1999,Amager Vest,Children below 18 years not living with parents,0 children,78
1221,2000,Amager Vest,Children below 18 years not living with parents,0 children,77
1222,2001,Amager Vest,Children below 18 years not living with parents,0 children,72
1223,2002,Amager Vest,Children below 18 years not living with parents,0 children,81
...,...,...,...,...,...
36795,2016,Østerbro,Single women,More than 3 children,21
36796,2017,Østerbro,Single women,More than 3 children,20
36797,2018,Østerbro,Single women,More than 3 children,21
36798,2019,Østerbro,Single women,More than 3 children,19


## Join genders

In [ ]:
cph_data_superset.display_dataframes()

In [112]:
df = cph_data.citizenship

display(df)

display(
    df.loc[df['Year'].isin([1980])
           & df['District'].isin(['1. Nord'])
           & df['Age'].isin(['0-9'])]
)

display(
    df.loc[df['Year'].isin([1980])
           & df['District'].isin(['1. Nord'])
           & df['Age'].isin(['0-9'])]
)

non_aggregation_columns = [column for column in df.columns
                           if column not in ['Sex', 'Number of people']]


df2 = (df
       .loc[df['Year'].isin([1980])
            & df['District'].isin(['1. Nord'])
            & df['Age'].isin(['0-9'])]
       .pivot_table(values='Number of people',
                    index=[column for column in df.columns
                           if column not in ['Sex', 'Number of people']],
                    columns='Sex',
                    aggfunc='first')
       .add_prefix('Number of ')
       .reset_index()
       )

# Delete the name of the index of columns.
df2.columns.name = ''
display(df2)

df3 = (
    df2
    .pivot_table(values=['Number of Men', 'Number of Women'],
                 index=[column for column in df2.columns
                        if column not in ['Citizenship', 'Number of Men', 'Number of Women']],
                 columns='Citizenship',
                 aggfunc='first')
    #     .add_prefix('Number of ')
    .reset_index()
)


display(df3)

,Year,Quarter,District,Sex,Age,Citizenship,District type,Number of people
0,1980,1,1. Nord,Men,0-9,Denmark,Polling area,265
1,1981,1,1. Nord,Men,0-9,Denmark,Polling area,278
2,1982,1,1. Nord,Men,0-9,Denmark,Polling area,265
3,1983,1,1. Nord,Men,0-9,Denmark,Polling area,249
4,1984,1,1. Nord,Men,0-9,Denmark,Polling area,249
...,...,...,...,...,...,...,...,...
157435,2016,4,Østerbro,Women,90+,Western countries,District,3
157436,2017,4,Østerbro,Women,90+,Western countries,District,4
157437,2018,4,Østerbro,Women,90+,Western countries,District,5
157438,2019,4,Østerbro,Women,90+,Western countries,District,6


,Year,Quarter,District,Sex,Age,Citizenship,District type,Number of people
0,1980,1,1. Nord,Men,0-9,Denmark,Polling area,265
2624,1980,1,1. Nord,Women,0-9,Denmark,Polling area,273
52480,1980,1,1. Nord,Men,0-9,Non-western countries,Polling area,19
55104,1980,1,1. Nord,Women,0-9,Non-western countries,Polling area,13
104960,1980,1,1. Nord,Men,0-9,Western countries,Polling area,15
107584,1980,1,1. Nord,Women,0-9,Western countries,Polling area,14


,Year,Quarter,District,Sex,Age,Citizenship,District type,Number of people
0,1980,1,1. Nord,Men,0-9,Denmark,Polling area,265
2624,1980,1,1. Nord,Women,0-9,Denmark,Polling area,273
52480,1980,1,1. Nord,Men,0-9,Non-western countries,Polling area,19
55104,1980,1,1. Nord,Women,0-9,Non-western countries,Polling area,13
104960,1980,1,1. Nord,Men,0-9,Western countries,Polling area,15
107584,1980,1,1. Nord,Women,0-9,Western countries,Polling area,14


,Year,Quarter,District,Age,Citizenship,District type,Number of Men,Number of Women
0,1980,1,1. Nord,0-9,Denmark,Polling area,265,273
1,1980,1,1. Nord,0-9,Non-western countries,Polling area,19,13
2,1980,1,1. Nord,0-9,Western countries,Polling area,15,14


Year Quarter District  Age District type Number of Men  \
Citizenship                                                 Denmark   
0            1980       1  1. Nord  0-9  Polling area           265   

                                                    Number of Women  \
Citizenship Non-western countries Western countries         Denmark   
0                              19                15             273   

                                                     
Citizenship Non-western countries Western countries  
0                              13                14

In [82]:
columns_to_remove = ['Age', 'Sex']
value columns



[column for column in df.columns
                   if column not in ['Sex', 'Number of people']]

# df['Number of people']

display(
    df
    .groupby(non_sex_columns, as_index=False)
    .sum()
)

,Year,Quarter,District,Age,Citizenship,District type,Number of people
0,1980,1,1. Nord,0-9,Denmark,Polling area,538
1,1980,1,1. Nord,0-9,Non-western countries,Polling area,32
2,1980,1,1. Nord,0-9,Western countries,Polling area,29
3,1980,1,1. Nord,10-19,Denmark,Polling area,606
4,1980,1,1. Nord,10-19,Non-western countries,Polling area,13
...,...,...,...,...,...,...,...
78715,2020,4,Østerbro,80-89,Non-western countries,District,28
78716,2020,4,Østerbro,80-89,Western countries,District,76
78717,2020,4,Østerbro,90+,Denmark,District,356
78718,2020,4,Østerbro,90+,Non-western countries,District,5
